# Define a SemantiLabeler class with the following methods:
* **predict(column,filename)**: given a column (column id + file name), predict semantic type of that column, using a pre-trained semantic labeling model
* **add_labeled_column(column,filename,attribute)**: given a labeled column (column id + file name + semantic label), add this labeled column to the "database" of labeled columns. If this column already exists in the "database", return an error.
* **replace_labeled_column(column,filename,attribute)**: given a labeled column, find it in the "database" of labeled columns, and replace its attribute with the new one. If there is nothing to replace, return an error.
* **validate_labels()**: Validate the labels file, by checking whether the column data can be extracted, and update the status of columns accordingly
* **update_training_examples()**: generate labeled examples from new labeled columns, and add them to the previous training set of labeled examples. This method updates the training set, in preparation for re-training the semantic classifier with train().
* **train()**: train a semantic labeling model on the current (updated) training set of examples
* **reset()**: reset the training set of examples, and the trained model. This makes the SemanticLabeler instance forget all it has learned.
* **status()**: return the current status of the semantic labeler: its directory (where the training sets and the trained model are stored), description of the trained model (including the random seed used), hyperparameters.

# Column status in the "database" of columns
The "knowlege base" of column labels is a dataframe with the following fields: 'col' (column id, assume that this is unique), 'file' (id of the file where the column is found), 'attribute' (semantic type of the column), and 'status'.
The 'status' field is used for internal purposes, and can have one of the following values:
* 'PROCESSED': this column has been processed, i.e., training examples have already been generated from it, and added to the training set of examples.
* 'NEW': this column has been added but has not yet been processed
* 'REPLACED': this column might have been processed before, but has been replaced since. It needs to be re-processed.
* 'NO_DATA': this column has not been found in the specified file.

# To do:
* **[DONE]** Implement handling new semantic types, on top of the previously known ones (open world)
* **[DONE]** Check for conflicting semantic types of the same columns
* **[DONE]** Finish the method for updating training examples.
* **[DONE]** implement train()
* implement reset()
* implement status()

In [1028]:
import os, csv
import gzip, pickle
import string
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier

In [1029]:
LOCATION = '../feature_engineering/repo/council_spending_data/data/'   # real estate data

In [1030]:
class Column(object):
    '''
    Data type to be passed to SemanticLabeler.predict
    '''
    def __init__(self, column, file):
        self.column = column
        self.file = file

class LabeledColumn(object):
    '''
    Data type to be passed to SemanticLabeler.label
    '''
    def __init__(self, column, file, label):
        self.column = column
        self.file = file
        self.label = label
        
class RequestHandler:
    def __init__(self):
        sl = SemanticLabeler(model_dir="SemanticLabeler")   # model_dir is the directory used internally by SemanticLabeler
        
    def send_to_semantic_labeler(request):
        '''
        request: JSON string, e.g. 
        '[{“col”: 1234, “filename”: “/some/other/path”, “label”: “Address"},
          {“col”: 9894, “filename”: “/some/path”, “label”: “Name"}]’
        '''
        reqs = json.loads(request)
        for req in reqs:
            sl.label(r["col"], r["filename"], r["label"])
        
    def update_semantic_labeler(): 
        sl.train()

In [1031]:
class SemanticLabeler(object):
    '''
    The task of SemanticLabeler class is to implement the methods for training and prediction of semantic data types, 
    given a list of data columns
    '''
#     ------ some internal functions: ------------------------------------------------------------------------
    def load_model(self, file_name):  #load the hyperparameters and the pre-trained model 
        save_file = gzip.open(file_name, 'rb')
        hp = pickle.load(save_file, encoding='bytes')      #load the hyperparameters
        model = pickle.load(save_file, encoding='bytes')   #load the model
        labels_dict = pickle.load(save_file, encoding='bytes') #load the targets_dict - the dict translating integer labels to attributes
        save_file.close()
        return (hp, model, labels_dict)
    
    def save_model(self,model,hp,labels_dict,file_name):  #save the hyperparameters and the model 
        save_file = gzip.open(file_name, 'wb')
        pickle.dump(hp, save_file, -1)      #save the hyperparameters
        pickle.dump(model, save_file, -1)   #save the model
        pickle.dump(labels_dict, save_file, -1)  # save the targets_dict - the dict translating integer labels to attributes
        save_file.close()
        
    def load_examples(self):
        examples_file = gzip.open(self.examples_path, 'rb')
        examples = pickle.load(examples_file,encoding='bytes')
        examples_file.close()
        return examples
    
    def save_examples(self, examples):
        examples_file = gzip.open(self.examples_path, 'wb')
        pickle.dump(examples,examples_file,-1)
        examples_file.close()
    
    def extract_column(self, col, file):
        df = pd.read_csv(file)
        if col in df:
            return df[col].values.tolist()
        else:
            print("ERROR: column",col,"is not found in file",file)
            return None
        
    def char_freq(self, text, freq=True, lowercase=True, entropy=True):  # extract character counts/frequencies in text (text is a list of strings)
        text = str(text)   # concatenate the elements of text into a single string
        all_chars = string.printable

        if lowercase: 
            text = text.lower()
        char_dic = {}
        if text is not '':
            for x in all_chars:
                char_dic[x] = float(text.count(x))/len(text) if freq else text.count(x)
        else:
            for x in all_chars:
                char_dic[x] = 0

        # add the information measure (negative entropy) of text, Sum(p(i) log(p(i)), i=1,...,n), where p(i) is the i-th character frequency, 
        # n is the number of possible characters in the "alphabet", i.e., number of possible states

        entr = 0
        max_entr = -np.log2(1./len(all_chars))   # maximum entropy, in bits per character, for a text made of all_chars
        if entropy:
            for x in string.printable:
                if freq:
                    p_x = char_dic[x]
                else:
                    p_x = float(char_dic[x])/len(text)
                if p_x > 0:
                    entr += - p_x * np.log2(p_x)   # entropy of text, in bits per character

        char_dic['_entropy_'] = entr/max_entr   # return the normalized entropy of text

        return char_dic
    
    def generate_examples(self,col,col_data,attribute=None,verbose=False):  #generate a matrix of examples from records, with number of records per example and number of examples per column defined in hp 
        if verbose: 
            print ('records_per_example = %i' %self.hp['records_per_example'])
            print ('examples_per_bucket = %i' %self.hp['examples_per_bucket'])
        if self.hp['entropy']:
            all_chars = list(string.printable) + ['_entropy_']
        else:
            all_chars = list(string.printable)
        # define the number of examples to generate:
        n_examples = self.hp['examples_per_bucket']

        examples = pd.DataFrame(index=range(n_examples), columns=['col']+['attribute']+['label']+all_chars)
        examples['col'][:] = col    # col is the column name
        if attribute is None: 
            examples['attribute'][:] = np.nan    # the attribute is not known
            examples['label'][:] = np.nan    # the label is not known
        else:
            examples['attribute'][:] = attribute
            examples['label'][:] = self.attributes_dict[attribute]  # int label corresponding to attribute
            
        for row in range(self.hp['examples_per_bucket']):
            if len(col_data)>0:
                sample = np.random.choice(col_data, size=self.hp['records_per_example'], replace=True)
            else:
                sample = ''   # empty string
            freq_vec = self.char_freq(sample, freq=self.hp['freq'], lowercase=self.hp['lowercase'], entropy=self.hp['entropy'])
            for char in all_chars:
                examples.set_value(row, char, freq_vec[char])

        return examples
    
#     ------ METHODS: --------------------------------------------------------------------------------

    def __init__(self,model_dir,seed=None,hp=None):
        
        if hp is not None: 
            self.hp = hp
        else:
            self.hp = {}
        self.seed = seed   # random seed to use for both example generation and random forest initialization
        self.path = os.path.join(os.getcwd(),model_dir)
        self.model_path = os.path.join(self.path, "rf_model.pkl.gz")
        self.labels_path = os.path.join(self.path, "labels.csv")
        self.examples_path = os.path.join(self.path, "examples.pkl.gz")
        self.updated_training_set = False
        if os.path.exists(self.path):
            print("The directory",self.path,"already exists.")
            print("Loading semantic classifier and hyperparameters from",self.model_path)
        else:
            print("Creating",self.path)
            os.makedirs(self.path)
            
        if os.path.exists(self.model_path):
                self.hp, self.classifier, self.labels_dict = self.load_model(self.model_path)  #load the model, hp parameters, and the labels_dict (dictionary of targets)
                print('\nHyperparameters overrided with the loaded hyperparameters:')
                print(self.hp)
        else:
            print("The model file",self.model_path,"is not found. Starting afresh.")
            self.classifier = None
            self.labels_dict = {}
            self.labels_dict[0] = 'UNKNOWN'   # this attribute should always be in the labels_dict
        self.attributes_dict = {v: k for k, v in self.labels_dict.items()}  # mapping from semantic types to int labels
        print('\nLoaded semantic classifier:\n', self.classifier,'\n')
        attributes = list(self.attributes_dict.keys())  # list of all known semantic types (attributes)

        print("\nChecking whether the labels file exists...", end='')
        if os.path.exists(self.labels_path):
            print("YES")
        else:
            print("NO. \ncreating a new labels file...")
            labels_file = csv.writer(open(self.labels_path, 'w'))
            labels_file.writerow(["col", "file", "attribute", "status"])   # create a header

        try:
            self.labels = pd.read_csv(self.labels_path, header=0)  # read the labels file
        except ValueError:
            #print('Error:', ValueError)
            self.labels = pd.DataFrame(columns=["col", "file", "attribute", "status"])  # create an empty dataframe

        print("\nChecking whether the training examples file exists...", end='')
        if os.path.exists(self.examples_path):
            print("YES")
        else:
            if self.classifier is not None:
                print("NO. \nYou still will be able to predict semantic types using this model, but you won't be able to update the model with new examples.")
            else:
                print("NO.")

    def predict(self, col, file, verbose=False):
        """
        Given col and file, predict the col's semantic type, using self.classifier
        """

        # Extract the column data:
        col_data = self.extract_column(col, file)
        if col_data is None:
            return None
        # Generate unlabeled examples from col_data:
        examples = self.generate_examples(col,col_data)
        if len(examples.index)==0:
            print ('predict: no examples to label, stopping.')
            return

        # predict soft labels (labels' posteriors) of examples:
        x = examples.iloc[:,3:].as_matrix()  # feature vectors of the examples
        if str(self.classifier.__class__) == "<class 'sklearn.ensemble.forest.RandomForestClassifier'>":
            y_score = self.classifier.predict_proba(x)
        else:
            print('predict: this classifier type',self.classifier.__class__ ,'is not yet implemented, stopping.')
            return 

        y_score_mean = np.mean(y_score, axis=0)  # take a mean of each column in y_score - i.e., mean soft label of all examples
        attribute = self.labels_dict[self.classifier.classes_[np.argmax(y_score_mean)]]  # semantic type with the highest posterior wins
        if verbose: 
            print("column",col,": semantic type is",attribute)

        return attribute
    
    def add_labeled_column(self, col, file, attribute, verbose=False):
        '''
        given a labeled column (column id + file name + semantic label),
        check for conflicts with previous labels, and if there are no conflicts,
        add this labeled column to the "database" of labeled columns.
        '''
        
        # add the new attribute to the attributes dictionary:
        if attribute not in self.attributes_dict.keys():  
            attr_label = max(self.attributes_dict.values())+1
            self.attributes_dict[attribute] = attr_label
            self.labels_dict[attr_label] = attribute
            
        # Check whether [col, file] already exists in self.labels:
        duplicate_labels = self.labels.loc[(self.labels['col'] == col) & (self.labels['file'] == file)]
        if duplicate_labels.shape[0]>0:
            # print('The label for',col,', ',file,'already exists; skipping.')
            return (False, 'duplicate label')    
        else:
            # this is a new label, add it to self.labels
            new_label = pd.DataFrame([[col, file, attribute, 'NEW']], columns=["col", "file", "attribute", "status"])
            self.labels = pd.concat([self.labels, new_label], ignore_index=True)
            return (True, 'label added')
        
        
    def replace_labeled_column(self, col, file, attribute, verbose=False):
        '''
        given a labeled column (column id + file name + semantic label),
        check for conflicts with previous labels, and if there are conflicts,
        replace the label of the corresponding column in the "database" of labeled columns.
        '''
        
        # add the new attribute to the attributes dictionary:
        if attribute not in self.attributes_dict.keys():  
            attr_label = max(self.attributes_dict.values())+1
            self.attributes_dict[attribute] = attr_label
            self.labels_dict[attr_label] = attribute
        
        # Check whether [col, file] already exists in self.labels:
        replaced_label_index = (self.labels['col'] == col) & (self.labels['file'] == file)
        if np.sum(replaced_label_index)>0:
            self.labels.loc[replaced_label_index] = [[col, file, attribute, 'REPLACED']]
            return (True, 'label replaced')    
        else:
            # this is a new label, nothing to replace in self.labels
            return (False, 'no label to replace')
        
    
    def validate_labels(self):
        '''
        Validate the labels file, by checking whether the column data can be extracted
        '''
        for index,row in self.labels.iterrows():
            col = row['col']
            file = row['file']
            attribute = row['attribute']
            status = row['status']
            
            if status == 'NEW' or status == 'REPLACED':
                col_data = self.extract_column(col, file)
                if col_data is None:
                    self.labels.ix[index,'status'] = 'NO_DATA'
                    
    
    def update_labels_file(self):
        '''
        Save the updated self.labels to the "database"
        '''
        self.labels.to_csv(self.labels_path, index=False)
        
    def update_training_examples(self):
        '''
        Update the training set of labeled examples, by generating examples from the newly labeled columns, 
        and appending them to the existing training set of labeled examples.
        This updates the training set, in preparation for re-training the semantic classifier with train().
        '''
        
        self.updated_training_set = False
        # import the training examples:
        if os.path.exists(self.examples_path):
            examples = self.load_examples()
        else:
            examples = None
    
        for index,row in self.labels.iterrows():
            col = row['col']
            file = row['file']
            attribute = row['attribute']
            status = row['status']
            
            if status == 'NEW' or status == 'REPLACED':
                col_data = self.extract_column(col, file)
                if col_data is not None:
                    new_examples = self.generate_examples(col,col_data,attribute)
                else:
                    new_examples = None
            
            if status == 'REPLACED' and (new_examples is not None):   # replace the examples previously generated from this column with new examples
                if examples is not None:
                    if examples[examples['col'] == col].shape[0]>0:  # if can find examples to replace, replace them
#                         print("Replacing examples:")
#                         print(examples.loc[examples['col'] == col])
#                         print("with new examples:")
#                         print(new_examples)
                        i2=0
                        for i1, row in examples[examples['col'] == col].iterrows():
                            examples.iloc[i1] = new_examples.iloc[i2]   # replace the previous examples for this col with the new examples generated from this col
                            i2 += 1
                    else:  # otherwise, just add the examples
                        examples = examples.append(new_examples, ignore_index=True)   # add new_examples to examples
                else:
                    examples = new_examples
                self.labels.ix[index,'status'] = 'PROCESSED'
                self.updated_training_set = True
                
            elif status == 'NEW' and (new_examples is not None):
                if examples is not None:
                    # examples = pd.concat([examples,new_examples], ignore_index=True)   # add new_examples to examples
                    examples = examples.append(new_examples, ignore_index=True)   # add new_examples to examples
                else:
                    examples = new_examples        
                self.labels.ix[index,'status'] = 'PROCESSED'
                self.updated_training_set = True
                
            
        if (self.updated_training_set) and (examples is not None): 
            self.update_labels_file()
            self.save_examples(examples)
        
        return self.updated_training_set
    
    def train(self):
        '''
        Train the semantic classifier, if the training examples have been updated (self.updated_training_set is True)
        '''
        def shuffle_examples(df):
            return df.iloc[np.random.permutation(len(df))]

        self.updated_classifier = False
        if self.updated_training_set:   # if the training set has been updated, re-train the classifier
            # import the training examples:
            if os.path.exists(self.examples_path):
                examples_train = self.load_examples()
                examples_train = shuffle_examples(examples_train)
                
                train_x = examples_train.iloc[:,3:].as_matrix()   # the feature vector starts from the 3rd column, columns 0-2 have bucket_id, attribute, and label 
                train_y = examples_train.iloc[:,2].as_matrix()    # column 2 contains integer labels of the examples.
                train_y = np.array([y for y in train_y])       # convert into a regular numpy array
                
                # create the classifier:
                rf = RandomForestClassifier(n_estimators=500, random_state=self.seed, n_jobs=4)
                # train the rf:
                rf.fit(train_x, train_y)
                self.save_model(model=rf,hp=self.hp,labels_dict=self.labels_dict,file_name=os.path.join(self.path,'rf_model.pkl.gz'))
                self.classifier = rf
                
                self.updated_classifier = True
                self.updated_training_set = False
            else:
                print("train: training examples file",self.examples_path,"not found, nothing to train on.")
                self.updated_classifier = False

        else:
            print("train: no new examples found, keeping the previous classifier.")
            self.updated_classifier = False
            
        return self.updated_classifier

### Specify hyperparameters:

In [1032]:
hp = {}
hp['examples_per_bucket'] = 10
hp['records_per_example'] = 100
hp['freq'] = True       # True for character frequencies, False for character counts
hp['lowercase'] = True  # Should the records be lower-cased when generating examples?
hp['entropy'] = True   # include text entropy in the feature vector, or not?

### Initialize a SemanticLabeler instance:

In [1033]:
sl = SemanticLabeler('SL_test', hp=hp)

Creating /Users/ytyshetskiy/Projects/Data_integration/code/open-data/SL_test
The model file /Users/ytyshetskiy/Projects/Data_integration/code/open-data/SL_test/rf_model.pkl.gz is not found. Starting afresh.

Loaded semantic classifier:
 None 


Checking whether the labels file exists...NO. 
creating a new labels file...

Checking whether the training examples file exists...NO.


#### The labels database of sl is stored in sl.labels. When starting from scratch, it's empty:

In [1034]:
sl.labels

,col,file,attribute,status


### Add some labeled columns to sl.labels: 
(note that the updated sl.labels is not saved to 'labels.csv' at this step)

In [1035]:
sl.add_labeled_column(col="Amount",file=os.path.join(LOCATION,'schema_12__council__royal-borough-of-greenwich__august_2013_payments_over_and_pound500_csv.csv'),attribute='expense_amount_paid')
sl.add_labeled_column(col="Payment Date",file=os.path.join(LOCATION,'schema_12__council__royal-borough-of-greenwich__august_2013_payments_over_and_pound500_csv.csv'),attribute='expense_payment_date')

(True, 'label added')

#### The labels database is now:

In [1036]:
sl.labels

,col,file,attribute,status
0,Amount,../feature_engineering/repo/council_spending_d...,expense_amount_paid,NEW
1,Payment Date,../feature_engineering/repo/council_spending_d...,expense_payment_date,NEW


### Update the training examples database:

In [1037]:
sl.update_training_examples()

True

Inspect the updated examples database: (this is for inspection purpose only)

In [1038]:
sl.load_examples()

,col,attribute,label,0,1,2,3,4,5,6,...,|,},~,,,,,,,_entropy_
0,Amount,expense_amount_paid,1,0.04681275,0.05179283,0.06474104,0.04880478,0.04581673,0.05876494,0.05677291,...,0,0,0,0.3934263,0,0.01394422,0,0,0,0.4651168
1,Amount,expense_amount_paid,1,0.306087,0.03362319,0.04985507,0.07304348,0.03652174,0.03652174,0.02898551,...,0,0,0,0.1733333,0,0.01391304,0,0,0,0.4993104
2,Amount,expense_amount_paid,1,0.04166667,0.06793478,0.05344203,0.04891304,0.04347826,0.04438406,0.04076087,...,0,0,0,0.4311594,0,0.01358696,0,0,0,0.4488105
3,Amount,expense_amount_paid,1,0.05590622,0.06672678,0.05229937,0.04147881,0.05500451,0.05320108,0.04418395,...,0,0,0,0.4238052,0,0.0135257,0,0,0,0.4507277
4,Amount,expense_amount_paid,1,0.03336339,0.04328224,0.06402164,0.05049594,0.04688909,0.05500451,0.04869252,...,0,0,0,0.4382326,0,0.0135257,0,0,0,0.4448019
5,Amount,expense_amount_paid,1,0.05520362,0.06696833,0.0479638,0.05158371,0.04343891,0.04524887,0.0479638,...,0,0,0,0.4461538,0,0.01357466,0,0,0,0.4395368
6,Amount,expense_amount_paid,1,0.04771372,0.0805169,0.06063618,0.04771372,0.0526839,0.04970179,0.04970179,...,0,0,0,0.3777336,0,0.0139165,0,0,0,0.4710917
7,Amount,expense_amount_paid,1,0.04809437,0.06079855,0.04537205,0.04537205,0.04809437,0.06442831,0.04537205,...,0,0,0,0.4392015,0,0.01361162,0,0,0,0.443592
8,Amount,expense_amount_paid,1,0.04360753,0.06342914,0.06937562,0.06739346,0.04658077,0.06144698,0.04955401,...,0,0,0,0.3478692,0,0.01387512,0,0,0,0.4844266
9,Amount,expense_amount_paid,1,0.04257246,0.06612319,0.05797101,0.03713768,0.04347826,0.04528986,0.04166667,...,0,0,0,0.4375,0,0.01268116,0,0,0,0.4449099


### Train the classifier sl.classifier, using the updated examples as a training set:
This trains the classifier (RF classifier for now), and saves it, along with the parameters, to 'rf_model.pkl.gz' file.

In [1039]:
sl.train()

True

### Predict the semantic attribute of a column, using the trained classifier:

In [1040]:
col="Amount"
file=os.path.join(LOCATION,'schema_12__council__royal-borough-of-greenwich__august_2013_payments_over_and_pound500_csv.csv')
col_data = sl.extract_column(col, file)
sl.predict(col,file, verbose=False)

'expense_amount_paid'

### Replace a label of a column: 
(can be used to change the attribute of a column to any other attribute including 'UNKNOWN')

In [1041]:
# sl.replace_labeled_column(col="Amount",file=os.path.join(LOCATION,'schema_12__council__royal-borough-of-greenwich__august_2013_payments_over_and_pound500_csv.csv'),attribute='expense_amount_paid_NEW')
sl.replace_labeled_column(col="Payment Date",file=os.path.join(LOCATION,'schema_12__council__royal-borough-of-greenwich__august_2013_payments_over_and_pound500_csv.csv'),attribute='expense_payment_date_NEW')

(True, 'label replaced')

Inspect the labels database after a label has been replaced:

In [1042]:
sl.labels

,col,file,attribute,status
0,Amount,../feature_engineering/repo/council_spending_d...,expense_amount_paid,PROCESSED
1,Payment Date,../feature_engineering/repo/council_spending_d...,expense_payment_date_NEW,REPLACED


Update the training examples, and inspect the examples database:

In [1043]:
sl.update_training_examples()
sl.load_examples()

,col,attribute,label,0,1,2,3,4,5,6,...,|,},~,,,,,,,_entropy_
0,Amount,expense_amount_paid,1,0.04681275,0.05179283,0.06474104,0.04880478,0.04581673,0.05876494,0.05677291,...,0,0,0,0.3934263,0,0.01394422,0,0,0,0.4651168
1,Amount,expense_amount_paid,1,0.306087,0.03362319,0.04985507,0.07304348,0.03652174,0.03652174,0.02898551,...,0,0,0,0.1733333,0,0.01391304,0,0,0,0.4993104
2,Amount,expense_amount_paid,1,0.04166667,0.06793478,0.05344203,0.04891304,0.04347826,0.04438406,0.04076087,...,0,0,0,0.4311594,0,0.01358696,0,0,0,0.4488105
3,Amount,expense_amount_paid,1,0.05590622,0.06672678,0.05229937,0.04147881,0.05500451,0.05320108,0.04418395,...,0,0,0,0.4238052,0,0.0135257,0,0,0,0.4507277
4,Amount,expense_amount_paid,1,0.03336339,0.04328224,0.06402164,0.05049594,0.04688909,0.05500451,0.04869252,...,0,0,0,0.4382326,0,0.0135257,0,0,0,0.4448019
5,Amount,expense_amount_paid,1,0.05520362,0.06696833,0.0479638,0.05158371,0.04343891,0.04524887,0.0479638,...,0,0,0,0.4461538,0,0.01357466,0,0,0,0.4395368
6,Amount,expense_amount_paid,1,0.04771372,0.0805169,0.06063618,0.04771372,0.0526839,0.04970179,0.04970179,...,0,0,0,0.3777336,0,0.0139165,0,0,0,0.4710917
7,Amount,expense_amount_paid,1,0.04809437,0.06079855,0.04537205,0.04537205,0.04809437,0.06442831,0.04537205,...,0,0,0,0.4392015,0,0.01361162,0,0,0,0.443592
8,Amount,expense_amount_paid,1,0.04360753,0.06342914,0.06937562,0.06739346,0.04658077,0.06144698,0.04955401,...,0,0,0,0.3478692,0,0.01387512,0,0,0,0.4844266
9,Amount,expense_amount_paid,1,0.04257246,0.06612319,0.05797101,0.03713768,0.04347826,0.04528986,0.04166667,...,0,0,0,0.4375,0,0.01268116,0,0,0,0.4449099


### Re-train the classifier:

In [1044]:
sl.train()

True

Note what happens when re-training again on the same training set:

In [1045]:
sl.train()

train: no new examples found, keeping the previous classifier.


False

### Predict a column's semantic attribute, using the updated classifier:

In [1046]:
# col="Amount"
col="Payment Date"
file=os.path.join(LOCATION,'schema_12__council__royal-borough-of-greenwich__august_2013_payments_over_and_pound500_csv.csv')
sl.predict(col,file, verbose=False)

'expense_payment_date_NEW'